In [ ]:
#!/usr/bin/env python3
#coding=utf-8
import time
from Arm_Lib import Arm_Device
import numpy as np
import cv2

# Create robot arm object
Arm = Arm_Device()
time.sleep(.1)
for i in range(3):
    Arm.Arm_RGB_set(255, 255, 255)
    time.sleep(.1)
    Arm.Arm_RGB_set(0,0, 0)
    time.sleep(.1)
time.sleep(1)

In [ ]:
import numpy as np
import cv2

def get_colors(img):
    color_map = {
        "red": [],
        "yellow": [],
        "green": [],
        "blue": [],
    }
    
    thresholds = {
        "red": {
            "lower": [0, 43, 200],
            "upper": [70, 60, 255]
        },
        "yellow": {
            "lower": [80, 210, 210],
            "upper": [150, 255, 255]
        },
        "green": {
            "lower": [15, 90, 46],
            "upper": [80, 150, 90]
        },
        "blue": {
            "lower": [120, 80, 20],
            "upper": [255, 140, 100]
        },
    }
    
    BGR = {}
    S = {}
    color_name={}
    img = cv2.resize(img, (160, 120), )
    # Convert color image to HSV
    HSV = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    # Take out the H, S, and V values of each row and column in turn and put them into the container.
    for i in range(0, 160):
        for j in range(0, 120):
            BGR[(i, j)] = [c for c in img[j, i]]
            S[(i, j)] = HSV[j, i][1]
            
    precision = 4
#     threshold = 0.9 * max(S.values())
    
    for point, color in BGR.items():
        
        if all(color[i] >= thresholds["red"]["lower"][i] for i in range(3)) and all(color[i] <= thresholds["red"]["upper"][i] for i in range(3)):
            color_map["red"].append(point)
        elif all(color[i] >= thresholds["yellow"]["lower"][i] for i in range(3)) and all(color[i] <= thresholds["yellow"]["upper"][i] for i in range(3)):
            color_map["yellow"].append(point)
        elif all(color[i] >= thresholds["green"]["lower"][i] for i in range(3)) and all(color[i] <= thresholds["green"]["upper"][i] for i in range(3)):
            color_map["green"].append(point)
        elif all(color[i] >= thresholds["blue"]["lower"][i] for i in range(3)) and all(color[i] <= thresholds["blue"]["upper"][i] for i in range(3)):
            color_map["blue"].append(point)
#     print(S)


    for color in ["red", "yellow", "green", "blue"]:
        points = [x for x in color_map[color]]
        if not points:
            color_map[color] = (-1, -1)
            continue
        
        saturation = [S[p] for p in points]
        
        avg_x = np.mean([p[1] for p in points])
        avg_y = np.mean([p[0] for p in points])
        color_map[color] = (avg_y, avg_x)
    
    return color_map, BGR

In [ ]:
import cv2
import ipywidgets.widgets as widgets
import threading
image_widget = widgets.Image(format='jpeg', width=600, height=500) #Set up the camera display component
color_widget = widgets.Image(format='jpeg', width=600, height=500)
image_container = widgets.HBox([image_widget, color_widget])
display(image_container) #Show camera components


color_centers = {
    "red": None,
    "yellow": None,
    "green": None,
    "blue": None
}

K = [
    [1113.62955, 0, 276.563928],
    [0, 1113.56836, 281.112193],
    [0, 0, 1.0]
]
    
dist = [-0.58734938,  3.10250603, -0.01497821,  0.01267831, -8.79105063]

#bgr8 to jpeg format
#image is 640x480
import enum
import cv2
def bgr8_to_jpeg(value, quality=75):
    return bytes(cv2.imencode('.jpg', value)[1])

def monitor():
    image = cv2.VideoCapture(1) #open camera
    if not image.isOpened():
        print("couldn't open the camera")
        return
    print("running")
    Arm.Arm_RGB_set(0, 255, 0)
    point_colors = {
        "red": (0,0,255),
        "yellow": (0,255,255),
        "green": (0,255,0),
        "blue": (255,0,0)
        }
    
    while True:
        ret, frame = image.read()
        frame = cv2.undistort(src=frame,
                                cameraMatrix=np.array(K),
                                distCoeffs=np.array(dist)
                               )
        color_image = np.ones((500, 600, 3), dtype=np.uint8) * 255
        try:
            color_map, BGR = get_colors(frame)
            for color, point in color_map.items():
                cv2.circle(frame, ( int(round(point[0] * 4)), int(round(point[1] * 4 )) ), 10, point_colors[color], -1)
                color_centers[color] = (int(round(point[0] * 4)), int(round(point[1] * 4 )) )
                print(color, ( int(round(point[0] * 4)), int(round(point[1] * 4 )) ))

            for point, clr in BGR.items():
                center = tuple([4*x for x in point])
                color = tuple([int(value) for value in clr])
                cv2.circle(color_image, center, 2, color, -1)
            
            image_widget.value = bgr8_to_jpeg(frame)
            color_widget.value = bgr8_to_jpeg(color_image)
            Arm.Arm_RGB_set(0, 255, 0)
            time.sleep(1)
            return
        except Exception as e:
            Arm.Arm_RGB_set(255, 0, 0)
            print(e)
            return
            
        time.sleep(1)

In [ ]:
Arm.Arm_serial_servo_write6(90, 130, 0, 0, 90, 90, 2000)

In [ ]:
camera_thread = threading.Thread(target=monitor)
camera_thread.start()

In [ ]:
block_locations = {}
blocks = {}

for clr, point in color_centers.items():
    # 2D image point (pixel coordinates)
    x, y = point
    if point[0] < 0:
        print(clr, "wasn't found")
        continue
        
        
    x_scaled = ((((x-300) * 13)/600) / 100) * -1
    y_scaled = ((((500 - y) * 0.867) * (21/500)) / 100.0) + 0.165
    
    pos = [round(x_scaled, 3), round(y_scaled, 3), 0.03]
    block_locations[clr] = pos;
    
    print(clr, "in camera coordinates:", [x,y])
    print(clr, "in base coordinates:", pos, "\n")
Arm.Arm_RGB_set(255,255,255)

In [ ]:
import math
from InverseKinematics import InverseKinematics
from scipy.spatial.transform import Rotation as R

r1 = np.array([ # 30
    [-0.00, -0.00, -1.00],
[-0.87, 0.50, 0.00],
[0.50, 0.87, -0.00]
])

r2 = np.array([ # 15
    [-0.00, -0.00, -1.00],
    [-0.97, 0.26, 0.00],
    [0.26, 0.97, -0.00] 
])

r3 = np.array([ #45
    [-0.00, -0.00, -1.00],
    [-0.71, 0.71, 0.00],
    [0.71, 0.71, -0.00]
])


def zrot(x, y):
    angle = math.atan2(x, y) * 1.4
    return int(round(90 + math.degrees(angle)))


def validJointAngles(q):
    if len(q) < 5:
        return False
    for i in range(4): # range is 180 for joints 1-4
        if q[i] < 0 or q[i] > 180:
            return False
        
    if q[4] < 0 or q[4] > 270:
        return False
    
    return True



for block, position in block_locations.items():
    print(block)
    
    p = [[x] for x in position]
    p[0] = [0]
    p[1] = [p[1][0] - 0.03]
    p[2] = [p[2][0] + 0.01]
    q = InverseKinematics(r1, p)
    q = [int(round(angle)) for angle in q]
    
    if not validJointAngles(q):
        q = InverseKinematics(r2, p)
        q = [int(round(angle)) for angle in q]
        
    if not validJointAngles(q):
        q = InverseKinematics(r3, p)
        q = [int(round(angle)) for angle in q]
        
    if not validJointAngles(q):
        print("out of range :(")
        print(q, "\n")
    else:
        q[0] = zrot(*position[:2])
        q[4] = 270
        print(q, "\n")
        blocks[block] = q

In [ ]:
from getpath import getPath

p = {
    "standby": [90, 130, 0, 0, 90],
    "top": [90, 80, 50, 50, 270],
    "above_stack": [45, 80, 50, 50, 270],
    "0": [45, 15, 35, 116, 270],
    "1": [45, 32, 27, 104, 270],
    "2": [45, 42, 32, 90, 270],
    "3": [45, 47, 43, 73, 270],
}

LEDs = {
    "red": (255, 0, 0),
    "yellow": (255, 255, 0),
    "green": (0, 255, 0),
    "blue": (0, 0, 255)
}

k_p = 0.6

# Define the function of clamping building blocks, enable=1：clamp，=0：release
def arm_clamp_block(enable, careful=False):
    t = 400
    if careful:
        t = 1000
    if enable == 0:
        Arm.Arm_serial_servo_write(6, 0, t)
    else:
        Arm.Arm_serial_servo_write(6, 140, t)
        time.sleep(0.5)
        
def correct_angle(angle, theoretical):
    if angle is not None:
        return angle
    
    if theoretical < 2:
        return 0
    if theoretical > 178:
        return 180
    else:
        return int(theoretical)

def move(start, end, grip = False):
    path = getPath(start, end)
    J6 = 0
    if grip:
        J6 = 140
        
    Arm.Arm_serial_servo_write6(*start, J6, 500)
    time.sleep(0.5)
    for i, joint_set in enumerate(path):
        Arm.Arm_serial_servo_write6(*joint_set, J6, 1000)
    
    time.sleep(1)
    q_actual = [Arm.Arm_serial_servo_read(j) for j in range(1,6)]
    q_actual = [correct_angle(angle, end[j]) for j, angle in enumerate(q_actual)]
    error = [int(end[j] - q_actual[j]) for j in range(5)]
    u = [k_p * error[j] for j in range(5)]
    q_corrected = [round(end[j] + u[j]) for j in range(5)]
    
    if validJointAngles(q_corrected):
        Arm.Arm_serial_servo_write6(*q_corrected, J6, 500)

In [ ]:
Arm.Arm_serial_servo_write6(*p["standby"], Arm.Arm_serial_servo_read(6), 1000)

In [ ]:
order = [
    "blue",
    "green",
    "red",
    "yellow",
]

move(p["standby"], p["top"])
Arm.Arm_RGB_set(*LEDs[order[0]])

for i, color in enumerate(order):
    Arm.Arm_RGB_set(*LEDs[order[i]])
    
    move(p["top"], blocks[color])
    print("moving from top to", blocks[color])
    time.sleep(1.25)
    arm_clamp_block(1)
    
    move(blocks[color], p["top"], True)
    time.sleep(1)
    move(p["top"], p["above_stack"], True)
    time.sleep(1.2)
    move(p["above_stack"], p[str(i)], True)
    print("stacking", color, "block")
    
    time.sleep(3)
    arm_clamp_block(0, True)
    time.sleep(1)
    
    move(p[str(i)], p["above_stack"])
    move(p["above_stack"], p["top"])
    Arm.Arm_RGB_set(0,0,0)

while True:
    Arm.Arm_RGB_set(*LEDs[order[0]])
    time.sleep(0.3)
    Arm.Arm_RGB_set(*LEDs[order[1]])
    time.sleep(0.3)
    Arm.Arm_RGB_set(*LEDs[order[2]])
    time.sleep(0.3)
    Arm.Arm_RGB_set(*LEDs[order[3]])
    time.sleep(0.3)
    